# Importacion de librerias y creacion de variables

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv

# Use the Trame backend for interactive inline plots
pv.set_jupyter_backend("trame")

n = 1 # Number of stacks of Miller indexes in each direction
c = 1.0 # Height of the unit cell "z" direction
a = 1.0 # Lattice constant

rhombohedral_angle = np.pi / 3 # radians

lattice_vectors = np.array([
    [a, 0, 0],
    [a * np.cos(rhombohedral_angle), a * np.sin(rhombohedral_angle), 0],
    [0, 0, c]
], dtype=float) # Rhombic cell

base_fractional = np.array([
    [0,   0,   0],
    [1/2, 1/2, 1],  # Este realmente no hace parte de la base pero lo pongo para que se vea el átomo de arriba, los de abajo son para brillouin
    [1/2, 1/2, 0], # Este si hace parte de la base
    [-1/2, 1/2, 0],
    [-1/2, -1/2, 0],
    [1/2, -1/2, 0]
], dtype=float) # Basis atoms positions

base = base_fractional[:3] @ lattice_vectors  # Coordenadas de los átomos en la celda unitaria

# Define Miller index to work with
Miller_indexes = np.stack(np.mgrid[0:n+1, 0:n+1, 0:n+1], axis=-1).astype(float)  # (n+1, n+1, n+1, 3) Miller indices grid
Miller_indexes = Miller_indexes.reshape(-1,3)[1:]  # Convertir a un array n^3 x 3

# Construcción de malla
I, J, K = np.mgrid[0:n+1, 0:n+1, 0:n+1]

# Combinación lineal increible
lattice = I[..., None] * lattice_vectors[0] + J[..., None] * lattice_vectors[1] + K[..., None] * lattice_vectors[2]
lattice = lattice.reshape(-1,3)[1:] # Al poner -1, numpy infiere la cantidad de puntos (2n+1)^3

print(np.shape(base))

(3, 3)


# Grafica de la celda

In [ ]:
def puntos_grafica(base, color, label, lattice_vectors):
    lengths = np.linalg.norm(lattice_vectors, axis=1)
    r = np.min(lengths) / 10  # Radio de las esferas (átomos)
    for i in range(np.size(base, 0)):  # Iterar sobre cada átomo en la base
        # Crear una esfera pequeña para representar un punto
        punto = pv.Sphere(radius=r, center=base[i])  # Centro en la posición del átomo
        # Añadir la esfera con un color específico y un nombre para la leyenda
        plotter.add_mesh(
            punto,
            color=color,
            label=label
        )

# Crear el plotter
plotter = pv.Plotter(notebook=True)

puntos_grafica(base, "green", "Nan", lattice_vectors)
puntos_grafica(lattice, "green", "Nan", lattice_vectors)

# Añadir la leyenda
plotter.add_legend()

# Mostrar
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:51322/index.html?ui=P_0x25e9e9a2c10_5&reconnect=auto" class="pyvis…

# Grafica celda reciproca

In [ ]:
# Calculate reciprocal lattice vectors
reciprocal_vectors = np.zeros((3, 3), dtype=float) # To store reciprocal lattice vectors
for i in range(3):
    reciprocal_vectors[i] = 2 * np.pi * np.cross(lattice_vectors[np.mod(i+1, 3)], lattice_vectors[np.mod(i+2, 3)])  / np.linalg.det(lattice_vectors)

lattice_reciprocal = I[..., None] * reciprocal_vectors[0] + J[..., None] * reciprocal_vectors[1] + K[..., None] * reciprocal_vectors[2]
lattice_reciprocal = lattice_reciprocal.reshape(-1,3)[1:] # Al poner -1, numpy infiere la cantidad de puntos (2n+1)^3

base_reciprocal = base_fractional @ reciprocal_vectors  # Coordenadas de los átomos en la celda unitaria

# Crear el plotter
plotter = pv.Plotter(notebook=True)

puntos_grafica(base_reciprocal[:3], "blue", "Nan", lattice_reciprocal)
puntos_grafica(lattice_reciprocal, "blue", "Nan", lattice_reciprocal)

# Añadir la leyenda
plotter.add_legend()

# Mostrar
plotter.show()

Widget(value='<iframe src="http://localhost:51322/index.html?ui=P_0x25e9e91b4d0_6&reconnect=auto" class="pyvis…

# Brillouin

In [ ]:
I, J, K = np.mgrid[-n:n+1, -n:n+1, -n:n+1]



# Combinación lineal increible
pts = I[..., None] * reciprocal_vectors[0] + J[..., None] * reciprocal_vectors[1] + K[..., None] * reciprocal_vectors[2]
pts = pts.reshape(-1,3) # Al poner -1, numpy infiere la cantidad de puntos (2n+1)^3

pts = np.concatenate(pts, base_reciprocal)

print(pts)

# Ordenamiento y eliminacion de puntos innecesarios
mag = np.linalg.norm(pts, axis=1)
order = np.argsort(mag)

pts_ord = pts[order][1:9] # Tomar los primeros 8 puntos (excluyendo el origen)

# Crear plotter
plotter = pv.Plotter()

hexagonal_planes = len(pts_ord) - 6

# Crear los planos
#Trucazo increible usando el parametro c_res
for i in range(hexagonal_planes): # Solo los planos hexagonales
    plane = pv.Disc(center=pts_ord[i,:]/2,      # punto central del plano
                     normal=pts_ord[i,:],   # vector normal al plano
                     inner=0.0, outer=0.355,   # tamaño del plano en direcciones radiales
                     r_res=1, c_res=6)  # densidad de subdivisión
    plotter.add_mesh(plane, color="aquamarine", opacity=0.5, show_edges=True)

for i in range(hexagonal_planes, len(pts_ord)): # Solo los planos faltantes, osea los mas lejanos y cuadrados
    plane = pv.Disc(center=pts_ord[i,:]/2,      # punto central del plano
                     normal=pts_ord[i,:],   # vector normal al plano
                     inner=0.0, outer=0.25,   # tamaño del plano en direcciones radiales
                     r_res=1, c_res=4)  # densidad de subdivisión
    plotter.add_mesh(plane, color="aquamarine", opacity=0.5, show_edges=True)

# Crear plotter# Mostrar
plotter.show()

[[ -6.28318531  -3.62759873  -6.28318531]
 [ -6.28318531  -3.62759873   0.        ]
 [ -6.28318531  -3.62759873   6.28318531]
 [ -6.28318531   3.62759873  -6.28318531]
 [ -6.28318531   3.62759873   0.        ]
 [ -6.28318531   3.62759873   6.28318531]
 [ -6.28318531  10.88279619  -6.28318531]
 [ -6.28318531  10.88279619   0.        ]
 [ -6.28318531  10.88279619   6.28318531]
 [  0.          -7.25519746  -6.28318531]
 [  0.          -7.25519746   0.        ]
 [  0.          -7.25519746   6.28318531]
 [  0.           0.          -6.28318531]
 [  0.           0.           0.        ]
 [  0.           0.           6.28318531]
 [  0.           7.25519746  -6.28318531]
 [  0.           7.25519746   0.        ]
 [  0.           7.25519746   6.28318531]
 [  6.28318531 -10.88279619  -6.28318531]
 [  6.28318531 -10.88279619   0.        ]
 [  6.28318531 -10.88279619   6.28318531]
 [  6.28318531  -3.62759873  -6.28318531]
 [  6.28318531  -3.62759873   0.        ]
 [  6.28318531  -3.62759873   6.28

Widget(value='<iframe src="http://localhost:51322/index.html?ui=P_0x25ebbf687d0_8&reconnect=auto" class="pyvis…